In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-24 15:38:22--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 10.16.90.117
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|10.16.90.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M   461KB/s    in 2.4s    

2024-02-24 15:38:26 (461 KB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))

1115394


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # string to integers
decode = lambda l: ''.join([itos[i] for i in l])  # integers to string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [12]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:100])

torch.Size([1115394]) <built-in method type of Tensor object at 0x7f13aad5c7c0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [14]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [23]:
torch.manual_seed(2024)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb)
print("targets:")
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")
        

inputs:
tensor([[56,  1, 44, 39, 58, 46, 43, 56],
        [56,  6,  1, 58, 46, 43,  1, 57],
        [43,  1, 34, 53, 50, 57, 41, 47],
        [46, 47, 57,  1, 54, 56, 53, 41]])
targets:
tensor([[ 1, 44, 39, 58, 46, 43, 56, 10],
        [ 6,  1, 58, 46, 43,  1, 57, 50],
        [ 1, 34, 53, 50, 57, 41, 47, 39],
        [47, 57,  1, 54, 56, 53, 41, 43]])
when input is [56] the target: 1
when input is [56, 1] the target: 44
when input is [56, 1, 44] the target: 39
when input is [56, 1, 44, 39] the target: 58
when input is [56, 1, 44, 39, 58] the target: 46
when input is [56, 1, 44, 39, 58, 46] the target: 43
when input is [56, 1, 44, 39, 58, 46, 43] the target: 56
when input is [56, 1, 44, 39, 58, 46, 43, 56] the target: 10
when input is [56] the target: 6
when input is [56, 6] the target: 1
when input is [56, 6, 1] the target: 58
when input is [56, 6, 1, 58] the target: 46
when input is [56, 6, 1, 58, 46] the target: 43
when input is [56, 6, 1, 58, 46, 43] the target: 1
when input is [56

In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(2024)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # num_embedding, embedding_dim
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # id shape: (B, T)
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)  # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.7138, grad_fn=<NllLossBackward0>)

Yr.YjvmHFQqBWq$?lP$PbdNM.
IT:aT:XAFhQMX:WMlcvTqeVtA:$?whM&KAXBk$JBh,zo!-fHP?$quhdMrn yNx-Vi3rXf?:Hxw


In [35]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [45]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.5512495040893555


In [48]:
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


By hobrm:
S:

Filane f d at f an dagice MNa thwedet ses! m.

Hoveig,
Gader
Hesthe d:

'sh btissthoperd g o bongusilkiremistomeshand, ound I:
An p'sscor; ber.
ARKES:
Ling
A:


Tw e we
S:
Wh'lownoue l s'Fot tadint' bicin yoowstheppay hano cor as s, ck con-bofrkn, d ies wars h stofes!
THaf amelan fey k
